In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import datetime
from IPython.display import display, HTML

2023-05-23 10:31:53.655693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 10:31:54.287254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-05-22 17:08:18.355656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-22 17:08:18.376680: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
df_weekly = pd.read_csv('../data/clean/df_weekly.csv')

In [4]:
data_admin_area = df_weekly.administrative_area_level_3.values

In [5]:
df_weekly.week.max()
lk_year_week = df_weekly[["administrative_area_level_3","year","week"]]
years = lk_year_week.year.unique()

min_data_per_lk = 99999999
max_data_per_lk = 0
data_per_lk_list = []
for lk in lk_year_week.administrative_area_level_3.unique():
    landk_len = len(lk_year_week[lk_year_week.administrative_area_level_3 == lk])
    # week_sum = 0
    # for y in years:
    #     week_sum += landk[lk_year_week.year == y].week.max()
    if landk_len < min_data_per_lk:
        min_data_per_lk = landk_len
    if landk_len > min_data_per_lk:
        max_data_per_lk = landk_len

    data_per_lk_list.append((lk,landk_len))
    #     min_weeks = week_sum
    # if week_sum > max_weeks:
    #     max_weeks = week_sum
# print(min_weeks,max_weeks)
print(min_data_per_lk,max_data_per_lk)

84 105


In [6]:
'''
Teile die In-Seqeuenz (Feature) in einzelne Sequenzen der Länge n_steps_in und
die Out-Sequenz (Target) in einzelne Sequenzen der Länge n_steps_out.

Beispiel: n_steps_in=1, n_steps_out=2
           Feature  | Target
    Seq 1:    1         -
        |     2         -
        |     3         6
        |     -         8       

@param overap 
Sollen sich die einzelnen Sequenzen überlappen?
    Standardmäßig überlappen sich die Daten der Sequenzen:
        1. Seq [1,2,3]
        2. Seq [2,3,4]
    mit overlap = False:
        1. Seq [1,2,3]
        2. Seq [4,5,6]

'''
def split_sequence(in_sequence, out_sequence, n_steps_in, n_steps_out, overlap=True):
    X, y = list(), list()
    step = 1 if overlap else n_steps_in
    left_over = 0 
    for i in range(0,len(in_sequence),step):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out -1
        # check if we are beyond the sequence
        if (out_end_ix > len(in_sequence)):
            left_over = len(in_sequence) - i
            break
   
        # gather input and output parts of the pattern     
        seq_x, seq_y = in_sequence[i:end_ix,:],  out_sequence[end_ix-1:out_end_ix,-1] #out_sequence[end_ix-1:out_end_ix,-1]
        X.append(seq_x)
        y.append(seq_y)

       
    return np.array(X), np.array(y), left_over

In [7]:
'''
Prüfe, ob in einer Sequenzen basieren auf den IN und OUT Steps 
LK übergreifenden Daten vorkommen.
Entferne Zeilen, welche nicht mehr für eine volle Sequenz verwendet werden können

BSP:
    1. LK Erfurt
    2. LK Erfurt
    3. LK Erfurt
    4. LK Ahrweiler
    5. LK Ahrweiler

'''
def prepare_df_for_seqeuncing(df,n_steps_in, n_steps_out):
    droped_indices = []
    df_len = len(df)-1
   
    for i in range(0,df_len):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out -1
        # check if we are beyond the sequence
        if (out_end_ix > df_len):
            break
        start_row = df.loc[i]
        end_row = df.loc[out_end_ix:out_end_ix]

        lk_of_seq_start = start_row.administrative_area_level_3
        lk_of_seq_end = end_row.administrative_area_level_3.iloc[0]

        # Wenn es nicht mehr genug "Zeilen" eines LKs gibt müssen diese entfernt werden
        if (lk_of_seq_start != lk_of_seq_end):
            droped_indices.append(i)          
    
    df = df.drop(df.index[droped_indices])
    return df

In [8]:

TIME_STEPS_IN = 5
TIME_STEPS_OUT = 1

print("DF Full:",len(df_weekly))

df_weekly_adjusted = prepare_df_for_seqeuncing(df_weekly,TIME_STEPS_IN, TIME_STEPS_OUT)

data =  np.array(df_weekly_adjusted[['confirmed', 'deaths' ,"recovered"
    ,"vaccines","people_vaccinated","people_fully_vaccinated"
    ,"school_closing","workplace_closing","cancel_events"
    ,"gatherings_restrictions","transport_closing"
    ,"stay_home_restrictions","internal_movement_restrictions"
    ,"internal_movement_restrictions","information_campaigns","testing_policy"
    ,"contact_tracing","facial_coverings"
    ,"vaccination_policy","elderly_people_protection","population"]])

print(data.shape)

new_data = np.array([[39919.42857142857, 1132.2857142857142, 38787.142857142855, 8832.0, 8832.0, 0.0, 3.0, 3.0, 2.0, 4.0, 1.0, 1.4285714285714286, -1.5714285714285714,
                      3.0, 2.0, 1.0, 1.1428571428571428, 2.0, 2.0, 2.0, 1841179.0]])  # Beispielhafte Eingangsdaten (Features)
# Vorbereitung der Eingangsdaten (Features) und der Zielvariablen
# features = data[:, :-1] 
# target = data[:, -1]

## Nimm Deaths als Target
target = data[:, 1]
features = data[:,:3] ## nim erstmal nur confirmed deaths
print("Features:",features.shape,"Target:",target.shape)

N_FEATURES = features.shape[-1]


# Daten normalisieren
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.reshape(-1, 1))
print("Features Scaled:",features_scaled.shape,"Target Scaled:",target_scaled.shape)
## SCALING Muss nach sequenzierung passieren, sonst passt die Normaliseirung nicht, wenn Werte fehlen

feature_seqs, target_seq, left_over =  split_sequence(features_scaled,target_scaled, TIME_STEPS_IN, TIME_STEPS_OUT, overlap=True)
df_weekly_adjusted = df_weekly_adjusted.drop(df_weekly_adjusted.index[-left_over:])
print("Features Seq.:",feature_seqs.shape,"Target Seq.:",target_seq.shape, "DF Adjust.:",len(df_weekly_adjusted))




train_size = int(len(feature_seqs) * 0.8)
test_size = len(feature_seqs) - train_size
print("Train Size: {}, Test Size: {}".format(train_size,test_size))
# train_features, test_features = features_scaled[:train_size], features_scaled[train_size:]
# train_target, test_target = target_scaled[:train_size], target_scaled[train_size:]


# feature -> target
# t1 -> t2
### ALTERNATIVE: Keras TimeSeriesGenerator https://keras.io/api/data_loading/timeseries/ 
# def create_lstm_dataset(features, target, time_steps):
#     X, y = [], []
#     for i in range(len(features) - time_steps):
#         X.append(features[i:(i+time_steps)])
#         y.append(target[i+time_steps])
#     return np.array(X), np.array(y)

# Das problem mit der input Größe >.<. Aktuell werden alle 5 daten aus den train data aneinandergereiht. Hier muss noch eine lösung her

# train_features_lstm, train_target_lstm = create_lstm_dataset(train_features, train_target, TIME_STEPS_IN)
# test_features_lstm, test_target_lstm = create_lstm_dataset(test_features, test_target, TIME_STEPS_IN)






DF Full: 40776
(38786, 21)
Features: (38786, 3) Target: (38786,)
Features Scaled: (38786, 3) Target Scaled: (38786, 1)
Features Seq.: (38782, 5, 3) Target Seq.: (38782, 1) DF Adjust.: 38782
Train Size: 31025, Test Size: 7757


In [144]:
# X, y = split_sequence(features_scaled,target_scaled, TIME_STEPS_IN, TIME_STEPS_OUT, admin_area=data_admin_area, overlap=True)
# print(X.shape,y.shape)

train_features_lstm, test_features_lstm = feature_seqs[:train_size], feature_seqs[train_size:]
train_target_lstm, test_target_lstm = target_seq[:train_size], target_seq[train_size:]

print("TRAIN:",train_features_lstm.shape,"TEST:",test_features_lstm.shape)
print(train_features_lstm[0,:,:2],train_target_lstm[0,:])
print(train_features_lstm[1,:,:2],train_target_lstm[1,:])

TRAIN: (31025, 5, 3) TEST: (7757, 5, 3)
[[0.00212008 0.0092743 ]
 [0.00231639 0.00972305]
 [0.00249426 0.01034276]
 [0.00261448 0.01059065]
 [0.0027581  0.01136849]] [0.01136849]
[[0.00231639 0.00972305]
 [0.00249426 0.01034276]
 [0.00261448 0.01059065]
 [0.0027581  0.01136849]
 [0.00289243 0.01184717]] [0.01184717]


In [145]:
model = Sequential()
model.add(LSTM(100, activation='relu', 
                return_sequences=True,
                input_shape=(TIME_STEPS_IN, N_FEATURES)))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(LSTM(100, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(Dense(128))
model.add(Dense(TIME_STEPS_OUT))
model.compile(optimizer='adam', loss='mse')

In [146]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



## Trainiere                                                
history = model.fit(
                    train_features_lstm, 
                    train_target_lstm, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(test_features_lstm, test_target_lstm),
                    callbacks=[tensorboard_callback])


loss = model.evaluate(test_features_lstm, test_target_lstm)

print('Test Loss:', loss)

Epoch 1/50
970/970 [==============================] - 8s 7ms/step - loss: 8.4475e-04 - val_loss: 7.8005e-04
Epoch 2/50
970/970 [==============================] - 6s 6ms/step - loss: 3.4755e-04 - val_loss: 6.8754e-04
Epoch 3/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3498e-04 - val_loss: 6.6400e-04
Epoch 4/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3022e-04 - val_loss: 7.1056e-04
Epoch 5/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3318e-04 - val_loss: 7.2080e-04
Epoch 6/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3218e-04 - val_loss: 7.5143e-04
Epoch 7/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3066e-04 - val_loss: 8.6269e-04
Epoch 8/50
970/970 [==============================] - 6s 6ms/step - loss: 3.2990e-04 - val_loss: 6.8858e-04
Epoch 9/50
970/970 [==============================] - 6s 6ms/step - loss: 3.3044e-04 - val_loss: 6.9634e-04
Epoch 10/50
970/970 [=======

In [156]:
INDEX = 5
TEST_INDEX_L = INDEX 
TEST_INDEX_H = INDEX+1
val_features = test_features_lstm[TEST_INDEX_L:TEST_INDEX_H]
val_target = test_target_lstm[TEST_INDEX_L:TEST_INDEX_H]

prediction = model.predict(val_features)


1/1 [==============================] - 0s 187ms/step


In [151]:
val_features[0,0,:]

array([0.08677858, 0.16065476, 0.08645905])

In [148]:
## * IN Sequence  | Time Steps die nur reingegeben werden
## * OUT Sequence | Time Steps die vom Netzwerk predicted werden
## * TIME_STEPS_OUT, da die Prediction für zwei Zeitschritte aus der Prediction des letzen Val-Wertes und des ersten komplett unbekannten besteht.
## -> ÜBerscheiden sich am Ende 
start_test_data = train_size
start_val_data = start_test_data +TEST_INDEX_L
end_val_data = (TEST_INDEX_H-TEST_INDEX_L)*TIME_STEPS_IN -1
end_prediciton_vals= TIME_STEPS_OUT-1

print("Val Input Features")
display(df_weekly_adjusted.loc[start_val_data:start_val_data+end_val_data])


## zur Kontrolle: de-scalierte Features
de_time_series_val_fatures = val_features.reshape((TEST_INDEX_H-TEST_INDEX_L)*TIME_STEPS_IN,N_FEATURES)
de_scaled_val_features = feature_scaler.inverse_transform(de_time_series_val_fatures)
## TODO: Heraus finden, warum hier immernoch die Zeile 32625 nimmt

de_scaled_target_features = target_scaler.inverse_transform(val_target.reshape(-1,1)).reshape(-1)
de_scaled_prediction = target_scaler.inverse_transform(prediction.reshape(-1,1)).reshape(-1)
print("Kontrolle: de-skalierte Features:")
print("  1.Zeile | confirmed: {}, recovered: {}".format(de_scaled_val_features[0,0],de_scaled_val_features[0,1]))
print("  letze Zeile | deaths: {} ".format(de_scaled_target_features[0]))

print("Val Excpedted Target")
display(df_weekly_adjusted.loc[start_val_data+end_val_data:start_val_data+end_val_data+end_prediciton_vals])

print("Prediction:")
print("deaths: {} ".format(de_scaled_prediction)) # TODO: Warum kommen hier 5 Werte raus, wenn die OUT_STEPS = 1 sind


Val Input Features


,administrative_area_level_3,year,week,confirmed,deaths,recovered,vaccines,people_vaccinated,people_fully_vaccinated,school_closing,...,stay_home_restrictions,internal_movement_restrictions,international_movement_restrictions,information_campaigns,testing_policy,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,population
31030,SK Brandenburg a.d.Havel,2021,28,2336.0,90.0,2246.0,133326.0,77393.0,55933.0,1.0,...,1.0,1.0,3.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0
31031,SK Brandenburg a.d.Havel,2021,29,2337.5,90.0,2247.5,144615.5,78507.5,66099.0,1.0,...,1.0,1.0,3.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0
31032,SK Brandenburg a.d.Havel,2021,30,2340.5,90.0,2250.5,150735.5,79523.5,71199.0,1.0,...,1.0,1.0,3.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0
31033,SK Brandenburg a.d.Havel,2021,31,2343.0,90.0,2253.0,152534.0,79934.0,72582.0,1.0,...,1.0,1.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0
31034,SK Brandenburg a.d.Havel,2021,32,2344.5,90.0,2254.5,157783.5,81453.5,76309.5,1.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0


Kontrolle: de-skalierte Features:
  1.Zeile | confirmed: 67573.0, recovered: 540.0
  letze Zeile | deaths: 557.0 
Val Excpedted Target


,administrative_area_level_3,year,week,confirmed,deaths,recovered,vaccines,people_vaccinated,people_fully_vaccinated,school_closing,...,stay_home_restrictions,internal_movement_restrictions,international_movement_restrictions,information_campaigns,testing_policy,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,population
31034,SK Brandenburg a.d.Havel,2021,32,2344.5,90.0,2254.5,157783.5,81453.5,76309.5,1.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,72124.0


Prediction:
deaths: [532.2243  539.7515  543.9816  552.62524 560.61053] 


In [149]:
df_weekly[df_weekly.deaths == 540]

,administrative_area_level_3,year,week,confirmed,deaths,recovered,vaccines,people_vaccinated,people_fully_vaccinated,school_closing,...,stay_home_restrictions,internal_movement_restrictions,international_movement_restrictions,information_campaigns,testing_policy,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,population
3270,LK Burgenlandkreis,2021,13,9972.857143,540.0,9432.857143,33511.142857,27643.428571,5867.714286,-3.0,...,-2.0,1.0,3.0,2.0,3.0,1.0,2.0,3.0,2.0,180190.0
13028,LK Leipzig,2022,23,96650.750000,540.0,96109.750000,498674.000000,180598.500000,184817.750000,0.0,...,1.0,0.0,1.0,2.0,3.0,2.0,2.0,5.0,2.0,257763.0
13029,LK Leipzig,2022,24,97364.400000,540.0,96823.400000,499193.200000,180603.000000,184848.800000,0.0,...,1.0,0.0,0.0,2.0,3.0,2.0,2.0,5.0,2.0,257763.0
13030,LK Leipzig,2022,25,98363.800000,540.0,97822.800000,499673.600000,180620.000000,184874.600000,0.0,...,1.0,0.0,0.0,2.0,3.0,2.0,2.0,5.0,2.0,257763.0
13031,LK Leipzig,2022,26,99630.000000,540.0,99089.000000,500055.800000,180637.000000,184903.600000,0.0,...,1.0,0.0,0.0,2.0,3.0,2.0,2.0,5.0,2.0,257763.0
28972,LK Wetteraukreis,2021,32,13538.000000,540.0,12998.000000,355686.285714,186387.000000,169299.285714,1.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,306460.0
28973,LK Wetteraukreis,2021,33,13655.666667,540.0,13115.666667,363028.500000,188395.166667,174633.333333,1.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,306460.0
28974,LK Wetteraukreis,2021,34,13828.285714,540.0,13288.285714,370276.714286,190984.142857,179290.428571,-3.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,306460.0
28975,LK Wetteraukreis,2021,35,14138.333333,540.0,13598.333333,375980.166667,193072.500000,182679.500000,-3.0,...,1.0,0.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,306460.0
32624,SK Erfurt,2022,28,67148.400000,540.0,66608.400000,557019.200000,209984.000000,207435.200000,0.0,...,1.0,0.0,0.0,2.0,3.0,2.0,2.0,5.0,2.0,213699.0
